<a href="https://colab.research.google.com/github/wangbxj1234/xformerforpt/blob/main/flowformer%E4%BB%A3%E7%A0%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

from torch import Tensor
from typing import Optional
from torch import nn


class Flow_Attention(nn.Module):
    # flow attention in normal version
    def __init__(self, in_planes):
        super(Flow_Attention, self).__init__()
        self.n_heads =8
        self.query_projection = nn.Linear(in_planes, in_planes)
        self.key_projection = nn.Linear(in_planes, in_planes)
        self.value_projection = nn.Linear(in_planes, in_planes)
        self.out_projection = nn.Linear(in_planes, in_planes)
        self.dropout = nn.Dropout(0.05)
        self.eps = 1e-6

    def kernel_method(self, x):
        return torch.sigmoid(x)

    def dot_product(self, q, k, v):
        kv = torch.einsum("hld,hlm->hdm", k, v)
        qkv = torch.einsum("hld,hdm->hlm", q, kv)
        return qkv

    def forward(self, pxo) -> torch.Tensor:
        p, x, o = pxo   
        PN=0;
        x_sub=[]
        p_sub=[]
        out=[]
        for offset in o:
           x_s=x[PN:int(offset),:]
           x_sub.append(x_s)
           tgt_len, embed_dim = x_s.size() ###(N,d) 
           #print(offset,tgt_len, embed_dim)
           PN=int(offset)
        ## input: (L or S) D; output: L D
        ## Note: queries, keys, values are not projected yet
        ## 1. Linear projection
           L=tgt_len
           S=tgt_len
           queries = self.query_projection(x_s).view(L, self.n_heads, -1)
           keys = self.key_projection(x_s).view(S, self.n_heads, -1)
           values = self.value_projection(x_s).view(S, self.n_heads, -1)
           queries = queries.transpose(0, 1)
           keys = keys.transpose(0, 1)
           values = values.transpose(0, 1)
           # 2. Non-negative projection
           queries = self.kernel_method(queries)
           keys = self.kernel_method(keys)
           ## 3. Flow-Attention
           # (1) Calculate incoming and outgoing flow
           sink_incoming = 1.0 / (torch.einsum("hld,hd->hl", queries + self.eps, keys.sum(dim=1) + self.eps))
           source_outgoing = 1.0 / (torch.einsum("hld,hd->hl", keys + self.eps, queries.sum(dim=1) + self.eps))
           # (2) conservation refine for source and sink
           conserved_sink = torch.einsum("hld,hd->hl", queries + self.eps,
                                      (keys * source_outgoing[:, :, None]).sum(dim=1) + self.eps)
           conserved_source = torch.einsum("hld,hd->hl", keys + self.eps,
                                        (queries * sink_incoming[:, :, None]).sum(dim=1) + self.eps)
           conserved_source = torch.clamp(conserved_source, min=-1.0, max=1.0)  # for stability
           # (3) Competition & Allocation
           sink_allocation = torch.sigmoid(conserved_sink * (float(queries.shape[1]) / float(keys.shape[1])))
           source_competition = torch.softmax(conserved_source, dim=-1) * float(keys.shape[1])
           # (4) dot product
           output = (self.dot_product(queries * sink_incoming[:, :, None],  # for value normalization
                              keys,
                              values * source_competition[:, :, None])  # competition
                * sink_allocation[:, :, None]).transpose(0, 1)  # allocation
           ## (5) Final projection
           output = output.reshape(L, -1)
           output = self.out_projection(output)
           output = self.dropout(output)

           out.append(output)
        #out=torch.IntTensor(out)
        out=torch.cat(out,dim=0)

        return out


In [ ]:
o=[ 838, 2088, 2317, 3567, 4541, 5561, 6427, 7077]
o=torch.IntTensor(o)
x = torch.rand(7077, 128)
p = torch.rand(7077, 3)

In [ ]:
planes=128
transformer2 = Flow_Attention(planes)
right_res = transformer2([p, x, o])
print(right_res.shape)

torch.Size([7077, 128])
